<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_07_1_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 7: LangChain: Agents**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 7 Material

* **Part 7.1: Introduction to LangChain Agents** [[Video]]() [[Notebook]](t81_559_class_07_1_agents.ipynb)
* Part 7.2: Understanding LangChain Agent Tools [[Video]]() [[Notebook]](t81_559_class_07_2_tools.ipynb)
* Part 7.3: LangChain Retrival and Search Tools [[Video]]() [[Notebook]](t81_559_class_07_3_search_tools.ipynb)
* Part 7.4: Constructing LangChain Agents [[Video]]() [[Notebook]](t81_559_class_07_4_more_agent.ipynb)
* Part 7.5: Custom Agents [[Video]]() [[Notebook]](t81_559_class_07_5_custom_agent.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [10]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_experimental duckduckgo-search langchainhub

Note: using Google CoLab


# 7.1: LangChain Agents

LangChain allows the use of [agents](https://python.langchain.com/v0.1/docs/modules/agents/), which revolutionize the way we interact with language models by transforming them into dynamic decision-makers. Unlike traditional chains, where each step is meticulously programmed, agents leverage the inherent reasoning capabilities of language models to choose their own paths. Imagine an intelligent system that not only understands the context but also determines the most effective sequence of actions to achieve a goal. This adaptability marks a significant departure from rigid, pre-defined workflows. By acting as a reasoning engine, the language model assesses each situation and decides which actions to take and in what order, offering unparalleled flexibility and efficiency. This chapter delves into the fascinating world of lang-chain agents, exploring how they harness the power of language models to autonomously navigate complex tasks, adapt to changing conditions, and deliver sophisticated solutions with minimal human intervention.

LangChain suppors multiple [agent types](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/). You will interface with these agents via two different methods:

* **Chat Models**: Designed to take in messages and output messages, requiring a specific prompting strategy for optimal performance.
* **LLMs**: Intended to take in a string and output a string, using a different prompting strategy to achieve the best results.

Some of the other important features for agent types include:

* **Supports Chat History**: Indicates if an agent can function as a chatbot, typically requiring advanced models.
* **Supports Multi-Input Tools**: Indicates if an agent can use tools requiring multiple inputs, often unsupported by earlier models.
* **Supports Parallel Function Calling**: Indicates if an agent can call multiple tools simultaneously, requiring more advanced models.

The LangChain agent types can be summarized as follows:

|Agent Type|Interface|Chat<br>History|Multi-Input<br>Tools|Parallel<br>Function Calling|When to use|
|--|--|--|--|--|--|
|Tool Calling|chat|✅|✅|✅|Used in this course, also tool-calling agents.
|XML|LLM|✅|||Use with XML models, such as Anthropic
|Structured Chat|chat|✅|✅||If you need to support tools with multiple inputs.
|JSON Chat|chat|✅|||If you are using a model good at JSON|
|ReAct|LLM|✅|||Complex reasoning|
|Self-Ask w/Search|LLM||||If you are using a simple model and only have one search tool|


In this course, we will focus primarily on tool-calling agents, which are designed to interact with external tools and APIs to perform specific tasks, such as data retrieval, processing, or executing actions based on user inputs. Tool-calling agents are highly versatile and are essential for building applications that require integration with various external systems. Besides tool-calling agents, there are two other important types: ReAct agents and Self-Ask with Search agents. ReAct agents combine reasoning and acting capabilities, making them suitable for complex decision-making and task automation scenarios where the agent needs to logically process information and take appropriate actions. Self-Ask with Search agents, on the other hand, are designed to enhance the language model's capabilities by integrating search functionalities, allowing the agent to retrieve up-to-date information from external sources. This makes them ideal for applications requiring accurate and current data retrieval, such as news aggregation, fact-checking, and research tools. Each type of agent serves a unique purpose, and understanding their use cases will enable you to select the appropriate agent for your specific application needs. We will also cover how to create an entirely custom agent.

## Key Components of Agents

Agents use a language model to decide on a sequence of actions. Unlike chains, where actions are predefined in the code, agents utilize a language model as a reasoning engine to determine the actions and their order.

* **AgentAction**: An AgentAction is a dataclass that defines the action an agent should take. It includes a tool property, specifying the name of the tool to be invoked, and a tool_input property, providing the input for that tool.

* **AgentFinish**: AgentFinish signifies the final result from an agent, ready to be returned to the user. It contains a return_values key-value mapping, typically holding an output key with a string representing the agent's response.

* **Intermediate Steps**: Intermediate steps represent previous agent actions and their corresponding outputs during the current run. These steps are crucial for informing future iterations, allowing the agent to track completed tasks. This is typed as a List (Tuple[AgentAction, Any), with observation being flexible and often a string.

## Agent Chain Structure

The agent is responsible for deciding the next step, usually powered by a language model, a prompt, and an output parser. Different agents use various prompting styles, input encoding methods, and output parsing techniques. For a comprehensive list of built-in agents, refer to the agent types section. Custom agents can also be created for more control.

* **Agent Inputs**: Inputs to an agent are key-value mappings, with intermediate_steps being the only required key, corresponding to the intermediate steps mentioned earlier. The PromptTemplate typically transforms these pairs into a format suitable for the LLM.

* **Agent Outputs**: The outputs can be the next action(s) to take or the final response to send to the user (AgentActions or AgentFinish). Concretely, this can be typed as Union (AgentAction, List[AgentAction], AgentFinish). The output parser converts the raw LLM output into one of these types.

## The AgentExecutor
The agent executor is the runtime that calls the agent, executes chosen actions, passes the action outputs back to the agent, and repeats the process. This runtime handles complexities such as:

* Selecting non-existent tools
* Managing tool errors
* Parsing tool invocation outputs
* Logging and observability at all levels (agent decisions, tool calls)

## Agent Tools
Tools are functions that an agent can invoke. The Tool abstraction consists of two components:

The input schema for the tool, informing the LLM of the necessary parameters.
The function to run, typically a Python function.

## Agent Toolkits
For common tasks, agents often require a set of related tools. LangChain provides toolkits—groups of 3-5 tools for specific objectives. For example, the GitHub toolkit includes tools for searching issues, reading files, and commenting. LangChain offers various toolkits to get started. For a full list, see the toolkits integrations section.

## Basic Agent Exmaple

Let's now take a look at a basic agent. An agent in this context is a system designed to perform specific tasks by leveraging tools and models. At a high level, this agent uses a language model to process inputs and make use of external tools to gather information. In our example, the agent is set up to answer questions using a search tool (DuckDuckGo) and a language model (OpenAI's ChatGPT). DuckDuckGo is used here because it is simple to integrate and does not require API keys. We will later explore how the agent is created and how it interacts with the tools and the language model to produce responses.

In [31]:
from re import VERBOSE
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

#MODEL = 'gpt-4-turbo'
MODEL = 'gpt-3.5-turbo'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is the currnet value of the DJIA?"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current value of the DJIA'}`


Dow Jones Industrial Average + Add to watchlist + Add an alert. DJI:DJI. Dow Jones Industrial Average. Actions. Add to watchlist; Add an alert; Price (USD) 39,411.21; Today's Change 260.88 / 0.67%; Shares traded 387.00m; ... Share price information may be rounded up/down and therefore not entirely accurate. Dow Jones Today: Get all information on the Dow Jones Index including historical chart, news and constituents. The Dow Jones Industrial Average ( DJINDICES:^DJI) is a stock index that tracks 30 of the largest U.S. companies. Created in 1896, it is one of the oldest stock indexes, and its performance is ... The observations for the Dow Jones Industrial Average represent the daily index value at market close. The market typically closes at 4 PM ET, except for holidays when it sometimes closes early. The Dow Jones Industrial Average provides a view of the US stock mark

{'input': 'What is the currnet value of the DJIA?',
 'output': 'The current value of the Dow Jones Industrial Average (DJIA) is 39,411.21.'}


The code sets up a basic agent to answer questions by integrating a language model and a search tool. Initially, it imports necessary modules and initializes the language model (ChatOpenAI) with specified parameters such as model type and temperature. The DuckDuckGoSearchRun tool is instantiated for performing web searches. A prompt template is pulled from a repository to guide the agent's responses. The agent is then created using the create_tool_calling_agent function, which combines the language model, tools, and prompt. Input data is provided to test the agent with a question about the president of the United States. The AgentExecutor class is used to manage the agent's execution, allowing it to process further queries, such as checking the current value of the DJIA, with verbosity enabled to provide detailed output during execution. This setup demonstrates the integration of a language model with a search tool to perform information retrieval and question-answering tasks.

# Module 7 Assignment

You can find the first assignment here: [assignment 7](https://github.com/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_class7.ipynb)